<a href="https://colab.research.google.com/github/OkanBagriacik/Evolutionary-Computating-Make-Up/blob/main/POGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Portfolio Optimization by using Genetic Algorithm**
Portfolio optimization method is one of the areas that attract the most attention and arouse curiosity in the financial field. Mathematicians and statisticians have regularly tried to find the ideal solution on this topic. This optimization method has been possible since the use of the genetic algorithm. A model was developed using specific stock market data.

# **Steps of How System Works**


1. Read the data and combine them into one dataframe.
2. Calculate the historical returns for 3 months, 6 months, 12 months, 24 months and 36 months for each of the stocks.
3. Define Gene (Scalar): A fraction of the total capital assigned to a stock.
4. Define Chromosome (1D Array): Set of genes i.e. fractions of total capital assigned to each stock.
         Check! Sum of each chromosome should be equal to 1.
5. Generate Initial Population (2D Array): A set of randomly generated chromosomes.
6. Fitness function (Define a Function): The Sharpe ratio, S, is a measure for quantifying the performance (Fitness) of the portfolio which works on "Maximisation of return (mean) and minimisation of risk (Variance) simultaneously" and is computed as follows:
       S = (µ − r)/σ

      Here µ is the return of the portfolio over a specified period or Mean portfolio return,
  r is the risk-free rate over the same period and 
  σ is the standard deviation of the returns over the specified period or Standard deviation of portfolio return.
Mean portfolio return = Mean Return * Fractions of Total Capital (Chromosome).
Risk-free rate = 0.0697 ( as per google)
Standard deviation of portfolio return = (chromosome * Standard deviation)**2 + Covariance * Respective weights in chromosome.

1. Select Elite Population (Define a Function): It filters the elite chromosomes which have highest returns, which was calculated in fitness function.
2. Mutation: A function that will perform mutation in a chromosome. Randomly we shall choose 2 numbers between 0, 5 and those elements we shall swap.
3. Crossover: Heuristic crossover or Blend Crossover uses the ﬁtness values of two parent chromosomes to ascertain the direction of the search. It moves from worst parent to best parent. The oﬀspring are created according to the equation:
     Off_spring A = Best Parent  + β ∗ ( Best Parent − Worst Parent)
     Off_spring B = Worst Parent - β ∗ ( Best Parent − Worst Parent)
         Where β is a random number between 0 and 1.
This crossover type is good for real-valued genomes.
4. Next Generation (define a Function): A function which does mutation,mating or crossover based on a probability and builds a new generation of chromosomes.
5. Iterate the process: Iterate the whole process till their is no change in maximum returns or for fixed number of iterations.

In [ ]:
import numpy as np
import pandas as pd
from functools import reduce

# In this part I am reading the data and combaining them into one dataframe

In [ ]:
files=['hdfc.csv','itc.csv','l&t.csv','m&m.csv','sunpha.csv','tcs.csv']
dfs=[]

for file in files:
    temp=pd.read_csv(file)
    temp.columns=['Date',file.replace('.csv','')]
    dfs.append(temp)

stocks = reduce(lambda left,right: pd.merge(left,right,on='Date'), dfs)
print(stocks.shape)
stocks.head()

(37, 7)


,Date,hdfc,itc,l&t,m&m,sunpha,tcs
0,June 2018,2108.05,266.05,1271.3,896.80,560.55,1847.20
1,May 2018,2136.15,271.60,1367.6,923.50,480.15,1744.80
2,Apr 2018,1944.60,281.45,1400.6,872.65,528.15,1765.70
3,Mar 2018,1891.45,255.90,1311.9,740.20,495.40,1424.65
4,Feb 2018,1883.80,265.10,1319.1,728.75,535.35,1519.13


# In this part I am calculating historical returns for preiod of mounth(3,6,12,24,36)

In [ ]:
def hist_return(months):
    ''' It calculates Stock returns for various months and returns a dataframe.
        Input: Months in the form of a list.
        Output: Historical returns in the form of a DataFrame. '''
    idx=[]
    df=pd.DataFrame()
    for mon in months:
        temp=(stocks.iloc[0,1:] - stocks.iloc[mon,1:])/(stocks.iloc[mon,1:])
        idx.append(str(mon)+'_mon_return')
        df=pd.concat([df, temp.to_frame().T], ignore_index=True)
    df.index=idx
    return df

In [ ]:
hist_stock_returns=hist_return([3,6,12,24,36])
hist_stock_returns

,hdfc,itc,l&t,m&m,sunpha,tcs
3_mon_return,0.114515,0.0396639,-0.0309475,0.211564,0.13151,0.296599
6_mon_return,0.125163,0.0112125,0.0114165,0.194062,-0.0179573,0.368094
12_mon_return,0.275866,-0.178478,0.129783,0.330899,0.0109107,0.562537
24_mon_return,0.792712,0.0842367,0.274461,0.255179,-0.265911,0.44833
36_mon_return,0.974847,0.266844,0.0696137,0.399938,-0.358785,0.447535


# Defining Genes

In [ ]:
gene = np.random.rand()
gene

0.9909349045414398

In [ ]:
import time
def gen_mc_grid(rows, cols, n, N):  # , xfname): generate monte carlo wind farm layout grids
        np.random.seed(seed=int(time.time()))  # init random seed
        layouts = np.zeros((n, rows * cols), dtype=np.int32)  # one row is a layout
        # layouts_cr = np.zeros((n*, 2), dtype=np.float32)  # layouts column row index
        positionX = np.random.randint(0, cols, size=(N * n * 2))
        positionY = np.random.randint(0, rows, size=(N * n * 2))
        ind_rows = 0  # index of layouts from 0 to n-1
        ind_pos = 0  # index of positionX, positionY from 0 to N*n*2-1
        # ind_crs = 0
        while ind_rows < n:
            layouts[ind_rows, positionX[ind_pos] + positionY[ind_pos] * cols] = 1
            if np.sum(layouts[ind_rows, :]) == N:
                # for ind in range(rows * cols):
                #     if layouts[ind_rows, ind] == 1:
                #         r_i = np.floor(ind / cols)
                #         c_i = np.floor(ind - r_i * cols)
                #         layouts_cr[ind_crs, 0] = c_i
                #         layouts_cr[ind_crs, 1] = r_i
                #         ind_crs += 1
                ind_rows += 1
            ind_pos += 1
            if ind_pos >= N * n * 2:
                print("Not enough positions")
                break
        # filename = "positions{}by{}by{}N{}.dat".format(rows, cols, n, N)
#         np.savetxt(lofname, layouts, fmt='%d', delimiter="  ")
        # np.savetxt(xfname, layouts_cr, fmt='%d', delimiter="  ")
        return layouts

def gen_mc_grid_with_NA_loc(rows, cols, n, N,NA_loc):  # , xfname): generate monte carlo wind farm layout grids
        np.random.seed(seed=int(time.time()))  # init random seed
        layouts = np.zeros((n, rows * cols), dtype=np.int32)  # one row is a layout, NA loc is 0

        layouts_NA= np.zeros((n, rows * cols), dtype=np.int32)  # one row is a layout, NA loc is 2
        for i in NA_loc:
            layouts_NA[:,i-1]=2

        # layouts_cr = np.zeros((n*, 2), dtype=np.float32)  # layouts column row index
        positionX = np.random.randint(0, cols, size=(N * n * 2))
        positionY = np.random.randint(0, rows, size=(N * n * 2))
        ind_rows = 0  # index of layouts from 0 to n-1
        ind_pos = 0  # index of positionX, positionY from 0 to N*n*2-1
        # ind_crs = 0
        N_count=0
        while ind_rows < n:
            cur_state=layouts_NA[ind_rows, positionX[ind_pos] + positionY[ind_pos] * cols]
            if cur_state!=1 and cur_state!=2:
                layouts[ind_rows, positionX[ind_pos] + positionY[ind_pos] * cols]=1
                layouts_NA[ind_rows, positionX[ind_pos] + positionY[ind_pos] * cols] = 1
                N_count+=1
                if np.sum(layouts[ind_rows, :]) == N:
                    ind_rows += 1
                    N_count=0
            ind_pos += 1
            if ind_pos >= N * n * 2:
                print("Not enough positions")
                break
        # filename = "positions{}by{}by{}N{}.dat".format(rows, cols, n, N)
#         np.savetxt(lofname, layouts, fmt='%d', delimiter="  ")
#         np.savetxt(loNAfname, layouts_NA, fmt='%d', delimiter="  ")
        # np.savetxt(xfname, layouts_cr, fmt='%d', delimiter="  ")
        return layouts,layouts_NA

In [ ]:
gen_mc_grid(5, 5, 100, 50)
gen_mc_grid_with_NA_loc(5, 5, 100, 50,range(10))

Not enough positions
Not enough positions


(array([[0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32), array([[2, 2, 2, ..., 1, 1, 2],
        [2, 2, 2, ..., 0, 0, 2],
        [2, 2, 2, ..., 0, 0, 2],
        ...,
        [2, 2, 2, ..., 0, 0, 2],
        [2, 2, 2, ..., 0, 0, 2],
        [2, 2, 2, ..., 0, 0, 2]], dtype=int32))

# Defining Chromosomes

In [ ]:
def chromosome(n):
    ''' Generates set of random numbers whose sum is equal to 1
        Input: Number of stocks.
        Output: Array of random numbers'''
    ch = np.random.rand(n)
    return ch/sum(ch)

child=chromosome(6)
print(child,sum(child))

[0.03747998 0.05876941 0.22879601 0.39510702 0.21345573 0.06639185] 1.0000000000000002


# Generating Population

In [ ]:
n=6 # Number of stocks = 6
pop_size=100 # initial population = 100

population = np.array([chromosome(n) for _ in range(pop_size)])


#Defining Fitness Funtion

In [ ]:
print(hist_stock_returns.info())
cols=hist_stock_returns.columns
hist_stock_returns[cols] = hist_stock_returns[cols].apply(pd.to_numeric, errors='coerce')
print(hist_stock_returns.info())

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, 3_mon_return to 36_mon_return
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   hdfc    5 non-null      float64
 1   itc     5 non-null      float64
 2   l&t     5 non-null      float64
 3   m&m     5 non-null      float64
 4   sunpha  5 non-null      float64
 5   tcs     5 non-null      float64
dtypes: float64(6)
memory usage: 280.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, 3_mon_return to 36_mon_return
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   hdfc    5 non-null      float64
 1   itc     5 non-null      float64
 2   l&t     5 non-null      float64
 3   m&m     5 non-null      float64
 4   sunpha  5 non-null      float64
 5   tcs     5 non-null      float64
dtypes: float64(6)
memory usage: 280.0+ bytes
None


In [ ]:
cov_hist_return=hist_stock_returns.cov()

print(cov_hist_return)

# For ease of calculations make covariance of same variable as zero.
for i in range(6):
    cov_hist_return.iloc[i][i]=0
    
cov_hist_return

            hdfc       itc       l&t       m&m    sunpha       tcs
hdfc    0.160272  0.045393  0.027916  0.024127 -0.079078  0.014362
itc     0.045393  0.025467 -0.000718  0.004381 -0.023178 -0.005554
l&t     0.027916 -0.000718  0.014206  0.002510 -0.013841  0.007330
m&m     0.024127  0.004381  0.002510  0.007412 -0.011042  0.005700
sunpha -0.079078 -0.023178 -0.013841 -0.011042  0.041781 -0.007211
tcs     0.014362 -0.005554  0.007330  0.005700 -0.007211  0.009923


,hdfc,itc,l&t,m&m,sunpha,tcs
hdfc,0.000000,0.045393,0.027916,0.024127,-0.079078,0.014362
itc,0.045393,0.000000,-0.000718,0.004381,-0.023178,-0.005554
l&t,0.027916,-0.000718,0.000000,0.002510,-0.013841,0.007330
m&m,0.024127,0.004381,0.002510,0.000000,-0.011042,0.005700
sunpha,-0.079078,-0.023178,-0.013841,-0.011042,0.000000,-0.007211
tcs,0.014362,-0.005554,0.007330,0.005700,-0.007211,0.000000


In [ ]:
mean_hist_return=hist_stock_returns.mean()
mean_hist_return


hdfc      0.456621
itc       0.044696
l&t       0.090865
m&m       0.278328
sunpha   -0.100047
tcs       0.424619
dtype: float64

In [ ]:
sd_hist_return=hist_stock_returns.std()
sd_hist_return

hdfc      0.400340
itc       0.159583
l&t       0.119189
m&m       0.086091
sunpha    0.204405
tcs       0.099615
dtype: float64

In [ ]:
def mean_portfolio_return(child):
    return np.sum(np.multiply(child,mean_hist_return))

In [ ]:
mean_portfolio_return(population[0])

0.21370971478392142

In [ ]:
def var_portfolio_return(child):
    part_1 = np.sum(np.multiply(child,sd_hist_return)**2)
    temp_lst=[]
    for i in range(6):
        for j in range(6):
            temp=cov_hist_return.iloc[i][j] * child[i] * child[j]
            temp_lst.append(temp)
    part_2=np.sum(temp_lst)
    return part_1+part_2

In [ ]:
var_portfolio_return(population[0])


0.026449187754280942

In [ ]:
rf= 0.0697

In [ ]:
def fitness_fuction(child):
    ''' This will return the Sharpe ratio for a particular portfolio.
        Input: A child/chromosome (1D Array)
        Output: Sharpe Ratio value (Scalar)'''
    return (mean_portfolio_return(child)-rf)/np.sqrt(var_portfolio_return(child))

In [ ]:
fitness_fuction(population[7])

1.8640323277871111

# Selecting Elite Population

In [ ]:
def Select_elite_population(population, frac=0.3):
    ''' Select elite population from the total population based on fitness function values.
        Input: Population and fraction of population to be considered as elite.
        Output: Elite population.'''
    population = sorted(population,key = lambda x: fitness_fuction(x),reverse=True)
    percentage_elite_idx = int(np.floor(len(population)* frac))
    return population[:percentage_elite_idx]

In [ ]:
[fitness_fuction(x) for x in population][:3]

[0.885493995259667, 1.0770890727954594, 1.6816714530736454]

# Mutation

In [ ]:
def mutation(parent):
    ''' Randomy choosen elements of a chromosome are swapped
        Input: Parent
        Output: Offspring (1D Array)'''
    child=parent.copy()
    n=np.random.choice(range(6),2)
    while (n[0]==n[1]):
        n=np.random.choice(range(6),2)
    child[n[0]],child[n[1]]=child[n[1]],child[n[0]]
    return child

In [ ]:
mutation(population[1]),population[1]

(array([0.2714113 , 0.2911401 , 0.0137925 , 0.18061458, 0.2139477 ,
        0.02909382]),
 array([0.18061458, 0.2911401 , 0.0137925 , 0.2714113 , 0.2139477 ,
        0.02909382]))

# Crossover

In [ ]:
def Heuristic_crossover(parent1,parent2):
    ''' The oﬀsprings are created according to the equation:
            Off_spring A = Best Parent  + β ∗ ( Best Parent − Worst Parent)
            Off_spring B = Worst Parent - β ∗ ( Best Parent − Worst Parent)
                Where β is a random number between 0 and 1.
        Input: 2 Parents
        Output: 2 Children (1d Array)'''
    ff1=fitness_fuction(parent1)
    ff2=fitness_fuction(parent2)
    diff=parent1 - parent2
    beta=np.random.rand()
    if ff1>ff2:
        child1=parent1 + beta * diff
        child2=parent2 - beta * diff
    else:
        child2=parent1 + beta * diff
        child1=parent2 - beta * diff
    return child1,child2

In [ ]:
for i in population[:30]:
    for j in population[:30]:
        print(Heuristic_crossover(i,j))

(array([0.30134941, 0.31386169, 0.17321496, 0.06173547, 0.06569375,
       0.08414473]), array([0.30134941, 0.31386169, 0.17321496, 0.06173547, 0.06569375,
       0.08414473]))
(array([ 0.08753299,  0.27362268, -0.10911568,  0.43306277,  0.32824541,
       -0.01334817]), array([ 0.394431  ,  0.3313791 ,  0.29612314, -0.099916  , -0.04860396,
        0.12658672]))
(array([0.12041439, 0.14427232, 0.05596073, 0.28558208, 0.12123343,
       0.27253706]), array([ 0.35629147,  0.36535857,  0.20881995, -0.00623696,  0.04882877,
        0.02693821]))
(array([ 2.34527418e-01, -1.24460577e-01,  4.74003088e-01,  3.04701278e-01,
        1.11335097e-01, -1.06304511e-04]), array([ 0.3283334 ,  0.49086454,  0.05175101, -0.03637874,  0.0472629 ,
        0.11816689]))
(array([ 0.20494572,  0.31417851, -0.0989508 ,  0.28378859,  0.06847392,
        0.22756406]), array([ 0.3394245 ,  0.31373656,  0.2807081 , -0.02596545,  0.0645957 ,
        0.02750059]))
(array([0.19516744, 0.12580769, 0.08581162, 0.146

In [ ]:
def Arithmetic_crossover(parent1,parent2):
    ''' The oﬀsprings are created according to the equation:
            Off spring A = α ∗ Parent1 + (1 −α) ∗ Parent2
            Off spring B = (1 −α) ∗ Parent1 + α ∗ Parent2
            
                Where α is a random number between 0 and 1.
        Input: 2 Parents
        Output: 2 Children (1d Array)'''
    alpha = np.random.rand()
    child1 = alpha * parent1 + (1-alpha) * parent2
    child2 = (1-alpha) * parent1 + alpha * parent2
    return child1,child2

In [ ]:
Arithmetic_crossover(population[2],population[3])

(array([0.23600564, 0.09494392, 0.2752792 , 0.20985025, 0.09629818,
        0.08762281]),
 array([0.20086221, 0.15336755, 0.16882565, 0.21434646, 0.10097453,
        0.16162359]))

# Creating Next Generation

In [ ]:
def next_generation(pop_size,elite,crossover=Heuristic_crossover):
    ''' Generates new population from elite population with mutation probability as 0.4 and crossover as 0.6. 
        Over the final stages, mutation probability is decreased to 0.1.
        Input: Population Size and elite population.
        Output: Next generation population (2D Array).'''
    new_population=[]
    elite_range=range(len(elite))
#     print(elite_range)
    while len(new_population) < pop_size:
        if len(new_population) > 2*pop_size/3: # In the final stages mutation frequency is decreased.
            mutate_or_crossover = np.random.choice([0, 1], p=[0.9, 0.1])
        else:
            mutate_or_crossover = np.random.choice([0, 1], p=[0.4, 0.6])
#         print(mutate_or_crossover)
        if mutate_or_crossover:
            indx=np.random.choice(elite_range)
            new_population.append(mutation(elite[indx]))
        else:
            p1_idx,p2_idx=np.random.choice(elite_range,2)
            c1,c2=crossover(elite[p1_idx],elite[p2_idx])
            chk=0
            for gene in range(6):
                if c1[gene]<0:
                    chk+=1
                else:
                    chk+=0
            if sum(chk)>0:
                p1_idx,p2_idx=np.random.choice(elite_range,2)
                c1,c2=crossover(elite[p1_idx],elite[p2_idx])
            new_population.extend([c1,c2])
    return new_population

# Iterating the process by Heuristic Crossover

In [ ]:
n=6 # Number of stocks = 6
pop_size=100 # initial population = 100

# Initial population
population = np.array([chromosome(n) for _ in range(pop_size)])

# Get initial elite population
elite = Select_elite_population(population)

iteration=0 
Expected_returns=0
Expected_risk=1

while (Expected_returns < 0.30 and Expected_risk > 0.0005) | iteration<=40:
    print('Iteration:',iteration)
    population
    elite = Select_elite_population(population)
    Expected_returns=mean_portfolio_return(elite[0])
    Expected_risk=var_portfolio_return(elite[0])
    print('Expected returns of {} with risk of {}\n'.format(Expected_returns,Expected_risk))
    iteration+=1


print('Portfolio of stocks after all the iterations:\n')
[print(hist_stock_returns.columns[i],':',elite[0][i]) for i in list(range(6))]

Iteration: 0
Expected returns of 0.14540568229103312 with risk of 0.00029478456066275166

Iteration: 1
Expected returns of 0.14540568229103312 with risk of 0.00029478456066275166

Iteration: 2
Expected returns of 0.14540568229103312 with risk of 0.00029478456066275166

Iteration: 3
Expected returns of 0.14540568229103312 with risk of 0.00029478456066275166

Iteration: 4
Expected returns of 0.14540568229103312 with risk of 0.00029478456066275166

Iteration: 5
Expected returns of 0.14540568229103312 with risk of 0.00029478456066275166

Iteration: 6
Expected returns of 0.14540568229103312 with risk of 0.00029478456066275166

Iteration: 7
Expected returns of 0.14540568229103312 with risk of 0.00029478456066275166

Iteration: 8
Expected returns of 0.14540568229103312 with risk of 0.00029478456066275166

Iteration: 9
Expected returns of 0.14540568229103312 with risk of 0.00029478456066275166

Iteration: 10
Expected returns of 0.14540568229103312 with risk of 0.00029478456066275166

Iteration

[None, None, None, None, None, None]

In [ ]:
print('Portfolio of stocks after all the iterations:\n')
[print(hist_stock_returns.columns[i],':',elite[0][i]) for i in list(range(6))]

print('\nExpected returns of {} with risk of {}\n'.format(Expected_returns,Expected_risk))

Portfolio of stocks after all the iterations:

hdfc : 0.0032709644684339274
itc : 0.2436122324833974
l&t : 0.15108573643796597
m&m : 0.08561075170834298
sunpha : 0.23598768932491043
tcs : 0.2804326255769493

Expected returns of 0.14540568229103312 with risk of 0.00029478456066275166



In [ ]:
fitness_fuction(elite[5])

2.5351269282808597